In [1]:
# settings
force_rerun = False
run_once = False # useful for debugging, just runs one sample

# imports
import os
import sys

# code for enabling this notebook to work within cursor
coralme_dir = '/home/chris/zuniga/coralme/' #'../'
sys.path.insert(0, coralme_dir)

import cobra
import coralme
import coralme.solver.solver
import coralme.builder.main
import coralme.core.model
from coralme.util.helpers import get_nlp, optimize
import pickle
from copy import deepcopy
import concurrent.futures

def run_growth_check(nlp, rxns_to_low_high, rxn_index_dct, met_index_dct, basis, save_path):
    temp_nlp = deepcopy(nlp)
    
    # set the reacitons to zero, save old values
    for rxn, (low, high) in rxns_to_low_high.items():
        temp_nlp.xl[rxn_index_dct[rxn]] = lambda x : low
        temp_nlp.xu[rxn_index_dct[rxn]] = lambda x : high

    # run the model
    sol, _ = optimize(rxn_index_dct, met_index_dct, temp_nlp, max_mu = 0.002, min_mu = 0.001, maxIter = 4,
		tolerance = 1e-4, precision = 'quad', verbose = True, basis = basis)

    # save the results
    pickle_out = open(save_path, 'wb')
    pickle.dump(sol, pickle_out)
    pickle_out.close()

carb_to_rxn = {
    'acetate' : 'EX_ac_e',
    'benzene' : 'EX_bz_e', # this is benzoic acid and not benzene, but I can't find benzene
    'glucose' : 'EX_glc__D_e', 
    'fructose' : 'EX_fru_e',
    'glycerol' : 'EX_glyc_e',
    'glutamate' : 'EX_glu__L_e',
    'sucrose' : 'EX_sucr_e',
    'toluene' : 'EX_tol_e',
}

def run_growth_checks_for_org(org_dir):
    # look to see if solution already exists
    print('running at '+str(org_dir))
    sols_dir = os.path.join(org_dir, 'outputs', 'aerobicity_sources_solutions')
    if not os.path.exists(sols_dir):
        os.mkdir(sols_dir)

    # make list of arguments and outputs needed
    file_to_args = {}
    for aero in ['aerobic', 'anaerobic']:
        for carbon in ['acetate', 'benzene', 'glucose', 'fructose', 'glycerol', 'glutamate', 'sucrose', 'toluene']:
            file_path = os.path.join(sols_dir, aero+'_'+carbon+'.pkl')
            if not os.path.exists(file_path):
                file_to_args.update({file_path : (aero, carbon)})
    if len(file_to_args) == 0:
        return()

    # load in model, generate nlp and basis
    path = os.path.join(org_dir, 'outputs', 'pathway_annotated_model.pkl')
    if not os.path.exists(path):
        return()
    ME_model = coralme.io.pickle.load_pickle_me_model(path)
    nlp = get_nlp(ME_model)
    rxn_index_dct = {r.id : ME_model.reactions.index(r) for r in ME_model.reactions}
    met_index_dct = {m.id : ME_model.metabolites.index(m) for m in ME_model.metabolites}
    
    # generate basis for later runs
    sol, basis = optimize(rxn_index_dct, met_index_dct, nlp, max_mu = 0.002, min_mu = 0.001, maxIter = 4,
    		tolerance = 1e-4, precision = 'quad', verbose = True, basis = None)
    
    # set up arguments and run solutions
    input_args = []
    run_checks = []
    for file, args in file_to_args.items():
        # let's set up the arguments
        rxns_to_low_high = {}
        if args[0] == 'anaerobic':
            rxns_to_low_high.update({'EX_o2_e' : (0, 1000)})
        else:
            rxns_to_low_high.update({'EX_o2_e' : (-17, 1000)})
        for carb, rxn in carb_to_rxn.items():
            if carb == args[1]:
                rxns_to_low_high.update({rxn : (-17, 1000)})
            else:
                rxns_to_low_high.update({rxn : (0, 1000)})

        # run our solution
        run_growth_check(nlp, rxns_to_low_high, rxn_index_dct, met_index_dct, basis, file)

    # finished!
    return()

In [ ]:
# set up arguments
base_dir = os.path.join(coralme_dir, 'species_files', 'Pseudomonas_files')
input_args = []
run_checks = []
for f in os.listdir(os.path.join(base_dir, 'individual_species')):
    if 'Reference' in f: continue
    org_dir = os.path.join(coralme_dir, 'species_files', 'Pseudomonas_files', 'individual_species', f)
    input_args.append(org_dir)
    run_checks.append(f)

# run
with concurrent.futures.ThreadPoolExecutor() as executor:
    # Start the load operations and mark each future with its URL
    future_to_check = {executor.submit(run_growth_checks_for_org, input_arg) : check for input_arg, check in zip(input_args, run_checks)}
    
    for future in concurrent.futures.as_completed(future_to_check):
        check = future_to_check[future]
        try:
            data = future.result()
        except Exception as exc:
            print('%r generated an exception: %s' % (check, exc))
        else:
            print(str(check)+' completed')


running at /home/chris/zuniga/coralme/species_files/Pseudomonas_files/individual_species/CP008896
running at /home/chris/zuniga/coralme/species_files/Pseudomonas_files/individual_species/CP053697
running at /home/chris/zuniga/coralme/species_files/Pseudomonas_files/individual_species/CP026386
running at /home/chris/zuniga/coralme/species_files/Pseudomonas_files/individual_species/CP069317
running at /home/chris/zuniga/coralme/species_files/Pseudomonas_files/individual_species/AE004091.2
running at /home/chris/zuniga/coralme/species_files/Pseudomonas_files/individual_species/CP014784
running at /home/chris/zuniga/coralme/species_files/Pseudomonas_files/individual_species/CP073105
running at /home/chris/zuniga/coralme/species_files/Pseudomonas_files/individual_species/CP065866
running at /home/chris/zuniga/coralme/species_files/Pseudomonas_files/individual_species/CP061848
running at /home/chris/zuniga/coralme/species_files/Pseudomonas_files/individual_species/CP012831
running at /home/c